In [ ]:
from typing import Collection

from pypfopt import EfficientFrontier, expected_returns, plotting, risk_models

from portfolio_optimization.engine.backtesting import Backtesting
import yfinance as yf

%matplotlib inline


In [ ]:
tickers: Collection[str] = ('BTC-USD', 'ETH-USD', 'ADA-USD', 'EIMI.L', 'SUSW.L', 'WSML.L')
df_prices = yf.download(' '.join(tickers), period="10y", interval="1d", group_by="ticker").stack(level=0).rename_axis(["Date", "Ticker"]).unstack(level=1).ffill().dropna()["Adj Close"]
df_prices.tail(-5)


In [ ]:
mu = expected_returns.capm_return(df_prices, frequency=365)

In [ ]:
S = risk_models.CovarianceShrinkage(df_prices, frequency=365).oracle_approximating()

In [ ]:
print("Correlation")
correlation_plot = plotting.plot_covariance(S, plot_correlation=True, dpi=500)
correlation_plot.figure.set_size_inches((12, 12))

In [ ]:
print("Covariance")
covariance_plot = plotting.plot_covariance(S, dpi=500)
covariance_plot.figure.set_size_inches((12, 12))

In [ ]:
backtest = Backtesting(df_prices, add_raw_tickers=True)

In [ ]:
ef = EfficientFrontier(mu, S)
ef.min_volatility()
weights = ef.clean_weights(rounding=2)
backtest.add_strategy(weights, name="ef_min_vol")
ef.portfolio_performance(verbose=True)

In [ ]:
for asset, weight in sorted(weights.items(), key=lambda x: x[1], reverse=True):
    if weight > 0:
        print(f"{asset}: {weight}")


In [ ]:
ef = EfficientFrontier(mu, S)
ef.max_sharpe()
weights = ef.clean_weights(rounding=2)
backtest.add_strategy(weights, name="ef_max_sharpe")
ef.portfolio_performance(verbose=True)

In [ ]:
for asset, weight in sorted(weights.items(), key=lambda x: x[1], reverse=True):
    if weight > 0:
        print(f"{asset}: {weight}")


In [ ]:
ef = EfficientFrontier(mu, S)
ef.max_quadratic_utility()
weights = ef.clean_weights(rounding=2)
backtest.add_strategy(weights, name="ef_max_quadratic")
ef.portfolio_performance(verbose=True)

In [ ]:
for asset, weight in sorted(weights.items(), key=lambda x: x[1], reverse=True):
    if weight > 0:
        print(f"{asset}: {weight}")

In [ ]:
results = backtest.run()
results.display()


In [ ]:
results.plot(freq="D", figsize=(20, 10), title="Capital gains")